In [61]:
import pandas as pd
import tensorflow as tf
# import numpy as np

# tf.set_random_seed(1)
# np.random.seed(1)

# 参数，超参数
batch_size = 50
lr = 0.1

def to_onehot(y):
    for i in range(len(y)):
        if y.iloc[i,0]==0:
            y.iloc[i,0] = 'y'
        else:
            y.iloc[i,0] = 'f'
    return pd.get_dummies(y,prefix=y.columns)

# 准备数据
train_data_set = pd.read_csv('train.csv')                   # shape:(4000,1602)
train_data = train_data_set.iloc[0:4000,1:1601]             # shape:(4000,1600)
train_labels = to_onehot(train_data_set.iloc[0:4000,[1601]])           # shape:(4000,1)
                                                            # shape:(4000,1)

# 测试数据不在该文件夹里，等哈改一下
# 先取一小批测试数据看看结果
test_data_set = pd.read_csv('test.csv')                 # shape:(3550,1601) 没有包含正确结果，和训练集不一样
test_data = test_data_set.iloc[0:1000,1:1601]           # shape:(1000,1600)
test_labels = to_onehot(
    pd.read_csv('sample_submit.csv').iloc[0:1000,[1]])          # shape:(1000,1) 一一对应

# 创建模型
tf_x = tf.placeholder(tf.float32,[None,40*40])          # 每批50个数据，此处暂时不管，为None
x_img = tf.reshape(tf_x,[-1,40,40,1])                   # (batch, height, width, channel)
tf_y = tf.placeholder(tf.float32,[None,2])

In [63]:
## CNN
conv1 = tf.layers.conv2d(           # shape:(40,40,1)
        inputs = x_img,
        filters = 10,               # 图像卷积后的深度
        kernel_size = 5,            # 扫描核5*5大小
        strides = 1,
        padding = 'same',
        activity_regularizer = tf.nn.relu
)                                   # shape:(40,40,10)
pool1 = tf.layers.max_pooling2d(
        inputs = conv1,
        pool_size = 2,
        strides = 2
)                                   # shape:(20,20,10)
conv2 = tf.layers.conv2d(
        inputs = pool1,
        filters = 20,
        kernel_size = 5,
        strides = 1,
        padding = 'same',
        activity_regularizer = tf.nn.relu
)                                   # shape:(20,20,20)
pool2 = tf.layers.max_pooling2d(
        inputs = conv2,
        pool_size = 2,
        strides = 2
)                                   # shape:(10,10,20)
# conv3 = tf.layers.conv2d(
#         inputs = pool2,
#         filters = 40,
#         kernel_size = 5,
#         strides = 1,
#         padding = 'same',
#         activity_regularizer = tf.nn.relu
# )                                   # shape:(10,10,40)

# pool3 = tf.layers.max_pooling2d(
#         inputs = conv3,
#         pool_size = 2,
#         strides = 2
# )                                   # shape:(5,5,40)

# flat_data = tf.reshape(pool3,[-1,5*5*40]) #(5*5*40, )      # 3层
flat_data = tf.reshape(pool2,[-1,10*10*20]) #(10*10*20, )  # 2层
# flat_data = tf.reshape(pool1,[-1,20*20*10]) #(20*20*10, )  #1层
output = tf.layers.dense(flat_data,2) #用于全连接层，一共输出两种分类个数

In [64]:
loss = tf.losses.softmax_cross_entropy(onehot_labels = tf_y,logits = output)
train_op = tf.train.AdamOptimizer(lr).minimize(loss)

In [65]:
# 计算精度
accuracy = tf.metrics.accuracy(
        labels = tf.argmax(tf_y,axis = 1),
        predictions = tf.argmax(output,axis = 1)
)[1]

In [66]:
# 重要步骤！！初始化
sess = tf.Session()
#初始化全局和本地变量
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess.run(init_op)

In [67]:
# 训练
for step in range(50000):
    b_x = train_data.iloc[step*batch_size:(step+1)*batch_size]
    b_y = train_labels.iloc[step*batch_size:(step+1)*batch_size]
    # _,loss_ = sess.run([train_op,loss],feed_dict = {tf_x:b_x,tf_y:b_y})
    sess.run(train_op,feed_dict = {tf_x:b_x,tf_y:b_y})
    if step % 200 ==0:
        accuracy_= sess.run(accuracy,feed_dict = {tf_x:test_data,tf_y:test_labels})
        print('Step:', step,'| test accuracy: %.8f' % accuracy_)
    # if step == 559:
    #     predictions_2 = tf.argmax(output,axis = 1)
    #     test_data_ = test_data.iloc[156:325]
    #     predictions_2_ ,output_= sess.run([predictions_2,output], feed_dict = {tf_x: test_data_})
    #     print('predictions_2:', predictions_2_,'output:',output_)
    if step == 49990:
        predictions_2 = tf.argmax(output,axis = 1)
        test_data_ = test_data.iloc[23:167]
        predictions_2_ ,output_= sess.run([predictions_2,output], feed_dict = {tf_x: test_data_})
        print('predictions_2:', predictions_2_,'output:',output_)

#当第一次运行完了后，就可以注释掉保存的网络了。
# 改参数可以在这里改，然后保存的模型位置又需要换一个，以免覆盖以前的。
#   保存网络
saver = tf.train.Saver()
save_path = saver.save(sess,"my_net/cnn_net.ckpt",write_meta_graph = False)
print("save_path:",save_path) #打印出来看看

Step: 0 | test accuracy: 0.50000000
Step: 200 | test accuracy: 0.50150001
Step: 400 | test accuracy: 0.50199997
Step: 600 | test accuracy: 0.50225002
Step: 800 | test accuracy: 0.50239998
Step: 1000 | test accuracy: 0.50250000
Step: 1200 | test accuracy: 0.50257140
Step: 1400 | test accuracy: 0.50262499
Step: 1600 | test accuracy: 0.50266665
Step: 1800 | test accuracy: 0.50269997
Step: 2000 | test accuracy: 0.50272727
Step: 2200 | test accuracy: 0.50274998
Step: 2400 | test accuracy: 0.50276923
Step: 2600 | test accuracy: 0.50278574
Step: 2800 | test accuracy: 0.50279999
Step: 3000 | test accuracy: 0.50281250
Step: 3200 | test accuracy: 0.50282353
Step: 3400 | test accuracy: 0.50283331
Step: 3600 | test accuracy: 0.50284213
Step: 3800 | test accuracy: 0.50285000
Step: 4000 | test accuracy: 0.50285715
Step: 4200 | test accuracy: 0.50286365
Step: 4400 | test accuracy: 0.50286955
Step: 4600 | test accuracy: 0.50287497
Step: 4800 | test accuracy: 0.50287998
Step: 5000 | test accuracy: 0.50

KeyboardInterrupt: 